In [1]:
# 환경 설정
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Any, Optional
import warnings
import time
from datetime import datetime
import torch
warnings.filterwarnings('ignore')

# GPU 설정 및 확인
def setup_gpu():
    """GPU 설정 및 확인"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_count = torch.cuda.device_count()
        gpu_name = torch.cuda.get_device_name(0)
        print(f"🚀 GPU 사용 가능: {gpu_name}")
        print(f"📊 GPU 개수: {gpu_count}개")
        print(f"🔧 사용할 디바이스: {device}")
        
        # GPU 메모리 정보 출력
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"💾 GPU 총 메모리: {total_memory:.2f}GB")
        
        return device
    else:
        print("⚠️ GPU를 사용할 수 없습니다. CPU를 사용합니다.")
        return torch.device("cpu")

# GPU 설정 실행
DEVICE = setup_gpu()

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 프로젝트 경로 설정
BASE_DIR = Path.cwd()
RERANKERS_DIR = BASE_DIR / "RAG_with_Various_Rerankers"
EMB_PATH = BASE_DIR / "output_chunks_with_embeddings.json"
QA_PATH = BASE_DIR / "real_estate_tax_QA.json"

# 패키지 경로 추가
sys.path.insert(0, str(RERANKERS_DIR))

print(f"🕐 평가 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"BASE_DIR: {BASE_DIR}")
print(f"RERANKERS_DIR: {RERANKERS_DIR}")
print(f"EMB_PATH exists: {EMB_PATH.exists()}")
print(f"QA_PATH exists: {QA_PATH.exists()}")

🚀 GPU 사용 가능: NVIDIA GeForce RTX 4080 Laptop GPU
📊 GPU 개수: 1개
🔧 사용할 디바이스: cuda
💾 GPU 총 메모리: 11.99GB
🕐 평가 시작 시간: 2025-09-17 17:03:22
BASE_DIR: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment
RERANKERS_DIR: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\RAG_with_Various_Rerankers
EMB_PATH exists: True
QA_PATH exists: True


In [2]:
# 각 카테고리별 대표 리랭커 1개씩 선별 (총 7개)
REPRESENTATIVE_RERANKER_MODULES = [
    # BM25 계열 대표 (CPU 기반, 빠르고 안정적)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 기본", "LegalRAGSystemBM25Rerank"),
    
    # CrossEncoder 계열 대표 (Cohere Reranker)
    ("CrossEncoder.RAG_Cohere_Rerank_FINAL", "Cohere", "LegalRAGSystemCohereRerank"),
    
    # BGE 계열 대표 (최적화된 버전 - 메모리 효율성 개선)(bge-reranker-v2-m3 모델)
    # ("CrossEncoder.BGE 계열.RAG_BGE_Base_Optimized_Rerank_FINAL", "BGE v2-m3", "LegalRAGSystemBGEBase"),
    
    # Embedding 계열 대표 (임베딩 기반)
    # ("Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL", "Embedding E5", "LegalRAGSystemEmbeddingE5"),
    
    # Hybrid 계열 대표 (결합 방법)
    ("Hybrid_Reranker.RAG_CombSum_Rerank_FINAL", "Hybrid CombSum", "LegalRAGSystemCombSum"),
    
    # LLM 계열 대표 (대화형 AI)
    ("LLM_Reranker.RAG_LLM_Rerank_FINAL", "LLM 기본", "LegalRAGSystemLLM"),
    
    # Rules 계열 (규칙 기반)
    ("Rules_Reranker.RAG_LegalRuleBoost_Rerank_FINAL", "Legal Rule Boost", "LegalRAGSystemLegalRuleBoost")
]

# 전체 리랭커 목록 (백업용)
ALL_RERANKER_MODULES = [
    # BM25 리랭커들 (5개)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 기본", "LegalRAGSystemBM25Rerank"),
    ("BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL", "BM25 CharNgram", "LegalRAGSystemBM25CharNgram"),
    ("BM25_Reranker.RAG_BM25_Kiwi_Rerank_FINAL", "BM25 Kiwi", "LegalRAGSystemBM25Kiwi"),
    ("BM25_Reranker.RAG_BM25_Regex_Rerank_FINAL", "BM25 Regex", "LegalRAGSystemBM25Regex"),
    ("BM25_Reranker.RAG_BM25_Stopword_Rerank_FINAL", "BM25 Stopword", "LegalRAGSystemBM25Stopword"),
    
    # CrossEncoder 리랭커들 (7개)
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL", "CrossEncoder MiniLM L6", "LegalRAGSystemMiniLML6"),
    ("CrossEncoder.RAG_CE_MiniLM_L12_Rerank_FINAL", "CrossEncoder MiniLM L12", "LegalRAGSystemMiniLML12"),
    ("CrossEncoder.RAG_CE_Electra_Rerank_FINAL", "CrossEncoder Electra", "LegalRAGSystemElectra"),
    ("CrossEncoder.RAG_CE_E5_Mistral_Rerank_FINAL", "CrossEncoder E5 Mistral", "LegalRAGSystemE5Mistral"),
    ("CrossEncoder.RAG_Cohere_Rerank_FINAL", "Cohere", "LegalRAGSystemCohereRerank"),
    ("CrossEncoder.RAG_MXBAI_Rerank_FINAL", "MXBAI", "LegalRAGSystemMXBAI"),
    
    # BGE 계열 (4개)
    ("CrossEncoder.BGE 계열.RAG_BGE_Base_Rerank_FINAL", "BGE Base", "LegalRAGSystemBGEBase"),
    ("CrossEncoder.BGE 계열.RAG_BGE_Large_Rerank_FINAL", "BGE Large", "LegalRAGSystemBGELarge"),
    ("CrossEncoder.BGE 계열.RAG_BGE_v2m3_Rerank_FINAL", "BGE v2 M3", "LegalRAGSystemBGEv2m3"),
    ("CrossEncoder.BGE 계열.RAG_Flashrank_BGEv2m3_Rerank_FINAL", "Flashrank BGE v2m3", "LegalRAGSystemFlashrankBGEv2m3"),
    
    # Embedding 리랭커들 (5개)
    ("Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL", "Embedding E5", "LegalRAGSystemEmbeddingE5"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_GTE_Rerank_FINAL", "Embedding GTE", "LegalRAGSystemEmbeddingGTE"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_MPNet_Rerank_FINAL", "Embedding MPNet", "LegalRAGSystemEmbeddingMPNet"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Paraphrase_Rerank_FINAL", "Embedding Paraphrase", "LegalRAGSystemEmbeddingParaphrase"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Stella_Rerank_FINAL", "Embedding Stella", "LegalRAGSystemEmbeddingStella"),
    
    # Hybrid 리랭커들 (4개)
    ("Hybrid_Reranker.RAG_CombSum_Rerank_FINAL", "Hybrid CombSum", "LegalRAGSystemCombSum"),
    ("Hybrid_Reranker.RAG_CombMNZ_Rerank_FINAL", "Hybrid CombMNZ", "LegalRAGSystemCombMNZ"),
    ("Hybrid_Reranker.RAG_RRF_Rerank_FINAL", "Hybrid RRF", "LegalRAGSystemRRF"),
    ("Hybrid_Reranker.RAG_WeightedSum_BM25_Embed_Rerank_FINAL", "Hybrid WeightedSum", "LegalRAGSystemWeightedSum"),
    
    # LLM 리랭커들 (3개)
    ("LLM_Reranker.RAG_LLM_Rerank_FINAL", "LLM 기본", "LegalRAGSystemLLM"),
    ("LLM_Reranker.RAG_LLM_Listwise_Rerank_FINAL", "LLM Listwise", "LegalRAGSystemLLMListwise"),
    ("LLM_Reranker.RAG_LLM_Pairwise_Rerank_FINAL", "LLM Pairwise", "LegalRAGSystemLLMPairwise"),
    
    # Rules 리랭커들 (1개)
    ("Rules_Reranker.RAG_LegalRuleBoost_Rerank_FINAL", "Legal Rule Boost", "LegalRAGSystemLegalRuleBoost")
]

# 사용할 리랭커 선택 (대표 리랭커 사용)
SELECTED_RERANKER_MODULES = REPRESENTATIVE_RERANKER_MODULES

print(f"🎯 선택된 대표 리랭커: {len(SELECTED_RERANKER_MODULES)}개")
print(f"📊 전체 리랭커: {len(ALL_RERANKER_MODULES)}개")

# 리랭커 유형별 분류
reranker_types = {
    'BM25': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'BM25' in name],
    'CrossEncoder': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'CrossEncoder' in name],
    'BGE': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'BGE' in name],
    'Embedding': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'Embedding' in name],
    'Hybrid': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'Hybrid' in name],
    'LLM': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'LLM' in name],
    'Rules': [name for module_path, name, class_name in SELECTED_RERANKER_MODULES if 'Rules' in name]
}

print("\n📋 선택된 대표 리랭커:")
for i, (module_path, name, class_name) in enumerate(SELECTED_RERANKER_MODULES, 1):
    category = ""
    if 'BM25' in name:
        category = "BM25 계열"
    elif 'CrossEncoder' in name:
        category = "CrossEncoder 계열"
    elif 'BGE' in name:
        category = "BGE 계열"
    elif 'Embedding' in name:
        category = "Embedding 계열"
    elif 'Hybrid' in name:
        category = "Hybrid 계열"
    elif 'LLM' in name:
        category = "LLM 계열"
    elif 'Rules' in name:
        category = "Rules 계열"
    
    print(f"  {i}. {name} ({category})")

print(f"\n📈 예상 평가 시간: 약 30-45분")
print(f"💾 예상 GPU 메모리 사용량: 6-8GB")

🎯 선택된 대표 리랭커: 5개
📊 전체 리랭커: 28개

📋 선택된 대표 리랭커:
  1. BM25 기본 (BM25 계열)
  2. Cohere ()
  3. Hybrid CombSum (Hybrid 계열)
  4. LLM 기본 (LLM 계열)
  5. Legal Rule Boost ()

📈 예상 평가 시간: 약 30-45분
💾 예상 GPU 메모리 사용량: 6-8GB


In [3]:
# 정답 데이터 로드 (전체 50개 사용)
with open(QA_PATH, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

print(f"📚 총 질문-답변 쌍: {len(qa_data)}개")

# 동일한 샘플 데이터를 사용하기 위한 설정
import random

# 재현 가능한 랜덤 시드 설정 (동일한 결과를 위해)
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 샘플 크기 설정 (테스트용으로 10개, 전체 평가용으로 50개)
SAMPLE_SIZE = 50  # 테스트용: 10개, 전체 평가용: 50개

if SAMPLE_SIZE >= len(qa_data):
    # 전체 데이터 사용
    sampled_data = qa_data
    print(f"📊 평가용 데이터: {len(sampled_data)}개 (전체 사용)")
else:
    # 동일한 샘플 데이터 생성 (시드 고정으로 재현 가능)
    sampled_data = random.sample(qa_data, SAMPLE_SIZE)
    print(f"📊 평가용 데이터: {len(sampled_data)}개 (샘플링, 시드: {RANDOM_SEED})")
    print(f"🔧 샘플 크기: {SAMPLE_SIZE}개")

# 샘플 데이터 미리보기
print(f"\n📋 샘플 데이터 미리보기 (처음 3개):")
for i, item in enumerate(sampled_data[:3]):
    print(f"  {i+1}. {item['question'][:50]}...")

# 동일한 샘플 데이터를 저장하고 로드하는 기능
def save_sample_data(sample_data, filename="sampled_qa_data.json"):
    """샘플 데이터를 파일로 저장"""
    sample_file = BASE_DIR / filename
    with open(sample_file, 'w', encoding='utf-8') as f:
        json.dump(sample_data, f, ensure_ascii=False, indent=2)
    print(f"💾 샘플 데이터 저장됨: {sample_file}")
    return sample_file

def load_sample_data(filename="sampled_qa_data.json"):
    """저장된 샘플 데이터를 로드"""
    sample_file = BASE_DIR / filename
    if sample_file.exists():
        with open(sample_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"📂 샘플 데이터 로드됨: {sample_file}")
        return data
    else:
        print(f"❌ 샘플 데이터 파일이 없습니다: {sample_file}")
        return None

# 현재 샘플 데이터 저장
sample_file = save_sample_data(sampled_data)

# 토픽별 분포 확인
topic_counts = {}
for item in sampled_data:
    topic = item['metadata']['topic']
    topic_counts[topic] = topic_counts.get(topic, 0) + 1

print("\n📋 토픽별 분포:")
for topic, count in sorted(topic_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {topic}: {count}개")

print(f"\n⏱️  전체 데이터 사용으로 예상 평가 시간: 약 45-60분")

📚 총 질문-답변 쌍: 50개
📊 평가용 데이터: 50개 (전체 사용)

📋 샘플 데이터 미리보기 (처음 3개):
  1. 종합부동산세에서 공제할 재산세는 어떻게 산정해야 하나요?...
  2. 2012년도 종합부동산세 공제 재산세는 어떻게 계산해야 하나요?...
  3. 종합부동산세의 공제 재산세는 어떤 공정시장가액비율을 적용해야 하나요?...
💾 샘플 데이터 저장됨: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\sampled_qa_data.json

📋 토픽별 분포:
  합산배제: 8개
  공제: 7개
  부속토지: 7개
  토지분류: 7개
  헌법: 6개
  신탁: 6개
  지분: 5개
  기타: 2개
  평등: 1개
  경정: 1개

⏱️  전체 데이터 사용으로 예상 평가 시간: 약 45-60분


In [4]:
# RAGAS 패키지 설치 및 import
try:
    import ragas
    from ragas import evaluate
    from ragas.metrics import (
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    )
    print("✅ RAGAS 패키지가 이미 설치되어 있습니다.")
except ImportError:
    print("📦 RAGAS 패키지를 설치합니다...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ragas[all]"])
    
    # 다시 import
    import ragas
    from ragas import evaluate
    from ragas.metrics import (
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    )
    print("✅ RAGAS 패키지 설치 및 import 완료")

✅ RAGAS 패키지가 이미 설치되어 있습니다.


In [5]:
# GPU 메모리 관리 및 최적화
import gc

def clear_gpu_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("🧹 GPU 메모리 정리 완료")
    gc.collect()

def get_gpu_memory_info():
    """GPU 메모리 정보 출력"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3  # GB
        cached = torch.cuda.memory_reserved() / 1024**3  # GB
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
        print(f"💾 GPU 메모리: {allocated:.2f}GB 사용 / {cached:.2f}GB 캐시 / {total:.2f}GB 총량")
        
        # 메모리 사용률 계산
        usage_percent = (allocated / total) * 100
        if usage_percent > 80:
            print(f"⚠️ GPU 메모리 사용률이 높습니다: {usage_percent:.1f}%")
            print("💡 GPU 메모리 정리를 권장합니다.")
        
        return allocated, cached, total
    else:
        print("⚠️ CUDA 사용 불가 - CPU 모드")
        return 0, 0, 0

def check_gpu_availability():
    """GPU 사용 가능 여부 확인"""
    if torch.cuda.is_available():
        print("✅ GPU 사용 가능")
        get_gpu_memory_info()
    else:
        print("❌ GPU 사용 불가 - PyTorch가 CUDA를 지원하지 않습니다.")
        print("💡 GPU 사용을 원한다면 CUDA 지원 PyTorch를 설치하세요:")
        print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

# GPU 상태 확인
check_gpu_availability()

# 리랭커 로드 및 초기화 함수
def load_reranker(module_path: str, class_name: str) -> Any:
    """리랭커 모듈을 동적으로 로드합니다."""
    try:
        # 로딩 전 GPU 메모리 상태 확인
        print("📊 로딩 전 GPU 메모리 상태:")
        get_gpu_memory_info()
        
        module = __import__(module_path, fromlist=[class_name])
        reranker_class = getattr(module, class_name)
        
        # GPU 메모리 부족 시 정리
        if torch.cuda.is_available():
            allocated, _, total = get_gpu_memory_info()
            if allocated / total > 0.7:  # 70% 이상 사용 시
                print("🧹 GPU 메모리 부족 - 정리 중...")
                clear_gpu_memory()
        
        reranker = reranker_class(embeddings_file=str(EMB_PATH))
        
        # 로딩 후 GPU 메모리 상태 확인
        print("📊 로딩 후 GPU 메모리 상태:")
        get_gpu_memory_info()
        
        return reranker
    except Exception as e:
        print(f"❌ {module_path} 로드 실패: {e}")
        # 실패 시에도 메모리 정리
        if torch.cuda.is_available():
            clear_gpu_memory()
        return None

# RAGAS 평가를 위한 데이터 준비 함수
def prepare_ragas_data(sampled_data: List[Dict], reranker: Any, reranker_name: str) -> List[Dict]:
    """RAGAS 평가를 위한 데이터를 준비합니다."""
    ragas_data = []
    
    for i, item in enumerate(sampled_data):
        try:
            question = item['question']
            ground_truth = item['ground_truth']
            ground_truth_contexts = item['ground_truth_contexts']
            
            # 리랭커로 문서 검색
            docs = reranker.retriever.invoke(question)
            
            # RAGAS 형식으로 변환
            ragas_item = {
                'question': question,
                'answer': ground_truth,  # 정답을 답변으로 사용
                'contexts': [doc.page_content for doc in docs],
                'ground_truth': ground_truth,
                'ground_truths': ground_truth_contexts
            }
            ragas_data.append(ragas_item)
            
        except Exception as e:
            print(f"❌ {reranker_name} - 질문 {i+1} 처리 실패: {e}")
            continue
    
    return ragas_data

# RAGAS 평가 실행 함수 (RAGAS 0.2.x 호환)
def evaluate_with_ragas(ragas_data: List[Dict], reranker_name: str) -> Dict[str, float]:
    """RAGAS 메트릭으로 리랭커를 평가합니다 (RAGAS 0.2.x 호환)."""
    try:
        # RAGAS 0.2.x 호환 데이터 준비
        try:
            from datasets import Dataset as HuggingFaceDataset
            
            # 데이터 구조 변환
            dataset_dict = {
                'question': [item['question'] for item in ragas_data],
                'answer': [item['answer'] for item in ragas_data],
                'contexts': [item['contexts'] for item in ragas_data],
                'ground_truth': [item['ground_truth'] for item in ragas_data],
                'ground_truths': [item['ground_truths'] for item in ragas_data]
            }
            
            eval_data = HuggingFaceDataset.from_dict(dataset_dict)
            print(f"✅ HuggingFace Dataset 생성 완료: {len(eval_data)}개 샘플")
            
        except Exception as e:
            print(f"⚠️ Dataset 생성 실패, DataFrame 사용: {e}")
            eval_data = pd.DataFrame(ragas_data)
            print(f"✅ DataFrame 생성 완료: {len(eval_data)}개 샘플")
        
        # RAGAS 평가 실행
        metrics = [
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy
        ]
        
        # RAGAS 0.2.x 호환 evaluate 호출
        try:
            result = evaluate(eval_data, metrics=metrics)
            
            # 결과 추출 (RAGAS 0.2.x 호환)
            scores = {}
            metric_names = ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']
            
            if hasattr(result, 'to_pandas'):
                # RAGAS 0.2.x 결과를 DataFrame으로 변환
                result_df = result.to_pandas()
                for metric in metric_names:
                    if metric in result_df.columns:
                        scores[metric] = float(result_df[metric].iloc[0])
                    else:
                        scores[metric] = 0.0
            else:
                # 다른 형식의 결과 처리
                for metric in metric_names:
                    if hasattr(result, metric):
                        scores[metric] = float(getattr(result, metric))
                    elif isinstance(result, dict) and metric in result:
                        scores[metric] = float(result[metric])
                    else:
                        scores[metric] = 0.0
            
            # 종합 점수 계산
            scores['overall_score'] = sum(scores.values()) / 4
            
            return scores
            
        except Exception as e:
            print(f"⚠️ RAGAS evaluate 실패: {e}")
            # 대안: 기본값 반환
            return {
                'context_precision': 0.0,
                'context_recall': 0.0,
                'faithfulness': 0.0,
                'answer_relevancy': 0.0,
                'overall_score': 0.0
            }
        
    except Exception as e:
        print(f"❌ {reranker_name} 평가 실패: {e}")
        return {
            'context_precision': 0.0,
            'context_recall': 0.0,
            'faithfulness': 0.0,
            'answer_relevancy': 0.0,
            'overall_score': 0.0
        }

print("🔧 평가 함수들이 준비되었습니다.")

✅ GPU 사용 가능
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량
🔧 평가 함수들이 준비되었습니다.


In [6]:
# GPU 사용을 위한 추가 설정
def configure_gpu_settings():
    """GPU 사용을 위한 최적화 설정"""
    if torch.cuda.is_available():
        # GPU 메모리 할당 방식 설정
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
        
        # GPU 메모리 캐시 설정
        torch.cuda.empty_cache()
        
        # GPU 메모리 최적화
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        
        print("✅ GPU 최적화 설정 완료")
        print(f"🔧 CUDA 버전: {torch.version.cuda}")
        print(f"🔧 cuDNN 버전: {torch.backends.cudnn.version()}")
        
        # GPU 메모리 정보 출력
        get_gpu_memory_info()
        
        return True
    else:
        print("⚠️ GPU를 사용할 수 없습니다.")
        return False

# GPU 설정 실행
gpu_available = configure_gpu_settings()

if gpu_available:
    print("\n🚀 GPU 사용 모드로 설정되었습니다!")
    print("💡 리랭커 로딩 시 GPU를 활용하여 더 빠른 성능을 기대할 수 있습니다.")
else:
    print("\n⚠️ CPU 모드로 실행됩니다.")
    print("💡 GPU 사용을 원한다면 CUDA 지원 PyTorch를 설치하세요.")

✅ GPU 최적화 설정 완료
🔧 CUDA 버전: 12.6
🔧 cuDNN 버전: 90701
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량

🚀 GPU 사용 모드로 설정되었습니다!
💡 리랭커 로딩 시 GPU를 활용하여 더 빠른 성능을 기대할 수 있습니다.


In [7]:
# 대표 리랭커 로드 및 평가 실행
print("🚀 대표 리랭커 로딩 시작...")
print("=" * 80)

# 초기 GPU 메모리 상태
get_gpu_memory_info()

all_rerankers = {}
success_count = 0
failed_count = 0

for i, (module_path, name, class_name) in enumerate(SELECTED_RERANKER_MODULES, 1):
    print(f"\n[{i}/{len(SELECTED_RERANKER_MODULES)}] 로딩 중: {name}...")
    start_time = time.time()
    
    try:
        # 메모리 사용량 확인
        allocated, cached, total = get_gpu_memory_info()
        
        # 메모리 사용량이 8GB 초과하면 정리
        if allocated > 8.0:
            print(f"⚠️  메모리 사용량 초과 ({allocated:.2f}GB > 8.0GB). 정리 중...")
            clear_gpu_memory()
            get_gpu_memory_info()
        
        reranker = load_reranker(module_path, class_name)
        load_time = time.time() - start_time
        
        if reranker:
            all_rerankers[name] = reranker
            success_count += 1
            print(f"✅ {name} 로드 성공 ({load_time:.2f}초)")
        else:
            failed_count += 1
            print(f"❌ {name} 로드 실패 ({load_time:.2f}초)")
            
    except Exception as e:
        failed_count += 1
        print(f"❌ {name} 로드 실패: {e}")
        clear_gpu_memory()  # 실패 시 메모리 정리

print(f"\n📊 로딩 결과:")
print(f"  ✅ 성공: {success_count}개")
print(f"  ❌ 실패: {failed_count}개")
print(f"  📈 성공률: {success_count/(success_count+failed_count)*100:.1f}%")

# 최종 GPU 메모리 상태
get_gpu_memory_info()

if success_count == 0:
    print("⚠️  로드된 리랭커가 없습니다. 평가를 중단합니다.")
else:
    print(f"\n🎯 {success_count}개 대표 리랭커로 평가를 진행합니다.")

🚀 대표 리랭커 로딩 시작...
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량

[1/5] 로딩 중: BM25 기본...
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량
📊 로딩 전 GPU 메모리 상태:
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량
💾 GPU 메모리: 0.00GB 사용 / 0.00GB 캐시 / 11.99GB 총량
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (BM25 기반 Re-ranker) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
📊 로딩 후 GPU 메모리 상태:
💾 GPU 메모리: 2.09GB 사용 / 2.10GB 캐시 / 11.99GB 총량
✅ BM25 기본 로드 성공 (23.95초)

[2/5] 로딩 중: Cohere...
💾 GPU 메모리: 2.09GB 사용 / 2.10GB 캐시 / 11.99GB 총량
📊 로딩 전 GPU 메모리 상태:
💾 GPU 메모리: 2.09GB 사용 / 2.10GB 캐시 / 11.99GB 총량
💾 GPU 메모리: 2.09GB 사용 / 2.10GB 캐시 / 11.99GB 총량
✅ 환경 설정 완료
🚀 RAG 시스

In [8]:
# RAGAS 버전 확인 및 호환성 체크
print("🔍 RAGAS 버전 및 호환성 확인")
print("=" * 60)

try:
    import ragas
    print(f"✅ RAGAS 버전: {ragas.__version__}")
    
    # RAGAS 버전에 따른 호환성 확인
    ragas_version = ragas.__version__
    print(f"📋 RAGAS 버전 정보: {ragas_version}")
    
    # 주요 메트릭 import 테스트
    from ragas import evaluate
    from ragas.metrics import (
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    )
    
    # RAGAS 0.2.x 버전용 추가 imports
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    
    print("✅ RAGAS 메트릭 import 성공")
    print("✅ RAGAS 0.2.x 버전 호환 imports 성공")
    
    # Dataset 클래스 존재 여부 확인 (RAGAS 0.2.x 버전 호환)
    USE_DATASET = False
    try:
        # RAGAS 0.2.x에서는 Dataset이 다르게 구현됨
        from ragas import Dataset
        print("✅ RAGAS Dataset 클래스 사용 가능")
        USE_DATASET = True
    except ImportError:
        try:
            # 대안: datasets 라이브러리 사용
            from datasets import Dataset as HuggingFaceDataset
            print("✅ HuggingFace Dataset 사용 가능")
            USE_DATASET = True
        except ImportError:
            print("⚠️ Dataset 클래스 사용 불가 - DataFrame 사용")
            USE_DATASET = False
    
    print(f"🔧 사용할 데이터 타입: {'Dataset' if USE_DATASET else 'DataFrame'}")
    
except ImportError as e:
    print(f"❌ RAGAS import 실패: {e}")
    print("💡 다음 명령어로 RAGAS를 설치하세요:")
    print("   pip install ragas[all]")

print("🔧 RAGAS 호환성 확인 완료")

🔍 RAGAS 버전 및 호환성 확인
✅ RAGAS 버전: 0.2.15
📋 RAGAS 버전 정보: 0.2.15
✅ RAGAS 메트릭 import 성공
✅ RAGAS 0.2.x 버전 호환 imports 성공
✅ HuggingFace Dataset 사용 가능
🔧 사용할 데이터 타입: Dataset
🔧 RAGAS 호환성 확인 완료


In [9]:
# RAGAS 0.2.x 버전 호환 개별 평가 함수
def evaluate_single_reranker_v2(reranker_name: str, reranker: Any, sampled_data: List[Dict]) -> Dict[str, float]:
    """개별 리랭커 RAGAS 평가 함수 (RAGAS 0.2.x 호환)"""
    print(f"\n🔍 {reranker_name} 개별 RAGAS 평가 시작...")
    print("=" * 60)
    
    # 데이터 준비
    start_time = time.time()
    ragas_data = []
    
    print(f"📋 {reranker_name} RAGAS 데이터 준비 중...")
    for i, item in enumerate(sampled_data):
        try:
            question = item['question']
            ground_truth = item['ground_truth']
            ground_truth_contexts = item['ground_truth_contexts']
            
            # 리랭커로 문서 검색
            docs = reranker.search(question, k=10)
            
            # RAGAS 형식으로 변환
            ragas_item = {
                'question': question,
                'answer': ground_truth,
                'contexts': [doc.page_content for doc in docs],
                'ground_truth': ground_truth,
                'ground_truths': ground_truth_contexts
            }
            ragas_data.append(ragas_item)
            
            if (i + 1) % 5 == 0:
                print(f"  진행률: {i+1}/{len(sampled_data)} ({(i+1)/len(sampled_data)*100:.1f}%)")
                
        except Exception as e:
            print(f"❌ {reranker_name} - 질문 {i+1} 처리 실패: {e}")
            continue
    
    prep_time = time.time() - start_time
    print(f"✅ {reranker_name} 데이터 준비 완료 ({prep_time:.2f}초, {len(ragas_data)}개 샘플)")
    
    if not ragas_data:
        print(f"❌ {reranker_name} - 평가할 데이터가 없습니다.")
        return {
            'context_precision': 0.0,
            'context_recall': 0.0,
            'faithfulness': 0.0,
            'answer_relevancy': 0.0,
            'overall_score': 0.0
        }
    
    print(f"🤖 {reranker_name} RAGAS 평가 실행 중...")
    
    try:
        # RAGAS 0.2.x 버전용 LLM과 Embeddings 설정
        try:
            from langchain_openai import ChatOpenAI
            from ragas.llms import LangchainLLMWrapper
            from ragas.embeddings import LangchainEmbeddingsWrapper
            
            # OpenAI LLM 설정 (평가용)
            llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o", temperature=0))
            
            # 기존 리랭커의 임베딩 모델 사용 (intfloat/multilingual-e5-large-instruct)
            # 리랭커에서 임베딩 모델을 추출하여 RAGAS용으로 래핑
            embeddings = None
            
            # 방법 1: 직접 embedding_model 속성 확인
            if hasattr(reranker, 'embedding_model') and reranker.embedding_model is not None:
                embeddings = LangchainEmbeddingsWrapper(reranker.embedding_model)
                print(f"✅ 기존 임베딩 모델 사용: {type(reranker.embedding_model).__name__}")
            
            # 방법 2: retriever를 통한 임베딩 모델 확인
            elif hasattr(reranker, 'retriever') and hasattr(reranker.retriever, 'embeddings'):
                embeddings = LangchainEmbeddingsWrapper(reranker.retriever.embeddings)
                print(f"✅ 기존 임베딩 모델 사용: {type(reranker.retriever.embeddings).__name__}")
            
            # 방법 3: vectorstore를 통한 임베딩 모델 확인
            elif hasattr(reranker, 'vectorstore') and hasattr(reranker.vectorstore, 'embedding_function'):
                embeddings = LangchainEmbeddingsWrapper(reranker.vectorstore.embedding_function)
                print(f"✅ 기존 임베딩 모델 사용: {type(reranker.vectorstore.embedding_function).__name__}")
            
            # 방법 4: 대안 - 기본 임베딩 모델 사용
            else:
                from langchain_openai import OpenAIEmbeddings
                embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
                print(f"⚠️ 기본 임베딩 모델 사용 (기존 임베딩 모델을 찾을 수 없음)")
                
                # 디버깅 정보 출력
                print(f"🔍 리랭커 속성 확인:")
                print(f"  - embedding_model: {hasattr(reranker, 'embedding_model')}")
                print(f"  - retriever: {hasattr(reranker, 'retriever')}")
                print(f"  - vectorstore: {hasattr(reranker, 'vectorstore')}")
                if hasattr(reranker, 'retriever'):
                    print(f"  - retriever.embeddings: {hasattr(reranker.retriever, 'embeddings')}")
                if hasattr(reranker, 'vectorstore'):
                    print(f"  - vectorstore.embedding_function: {hasattr(reranker.vectorstore, 'embedding_function')}")
            
            print(f"✅ LLM 및 Embeddings 설정 완료")
            
        except Exception as e:
            print(f"⚠️ LLM/Embeddings 설정 실패: {e}")
            llm = None
            embeddings = None
        
        # RAGAS 평가를 위한 데이터 준비
        print(f"📊 RAGAS 평가 데이터 준비 중...")
        
        # HuggingFace Dataset 사용 (RAGAS 0.2.x 호환)
        try:
            from datasets import Dataset as HuggingFaceDataset
            
            # 데이터 구조 변환
            dataset_dict = {
                'question': [item['question'] for item in ragas_data],
                'answer': [item['answer'] for item in ragas_data],
                'contexts': [item['contexts'] for item in ragas_data],
                'ground_truth': [item['ground_truth'] for item in ragas_data],
                'ground_truths': [item['ground_truths'] for item in ragas_data]
            }
            
            eval_data = HuggingFaceDataset.from_dict(dataset_dict)
            print(f"✅ HuggingFace Dataset 생성 완료: {len(eval_data)}개 샘플")
            
        except Exception as e:
            print(f"⚠️ Dataset 생성 실패, DataFrame 사용: {e}")
            eval_data = pd.DataFrame(ragas_data)
            print(f"✅ DataFrame 생성 완료: {len(eval_data)}개 샘플")
        
        # RAGAS 평가 실행
        from ragas.metrics import (
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy
        )
        
        metrics = [
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy
        ]
        
        print(f"🔍 RAGAS 메트릭 계산 중...")
        
        # RAGAS 0.2.x 버전 호환 evaluate 호출
        try:
            # RAGAS 0.2.x에서는 metrics를 리스트로 전달하고, llm과 embeddings를 별도로 설정
            if llm and embeddings:
                result = evaluate(
                    eval_data,
                    metrics=metrics,
                    llm=llm,
                    embeddings=embeddings
                )
            else:
                # LLM/Embeddings 없이 평가 시도 (기본 설정 사용)
                result = evaluate(eval_data, metrics=metrics)
            
            print(f"✅ RAGAS 평가 완료")
            
            # 결과 추출 (RAGAS 0.2.x 호환)
            print(f"📊 RAGAS 평가 결과 처리 중...")
            
            scores = {}
            metric_names = ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']
            
            # RAGAS 0.2.x 결과 형식에 맞게 추출
            if hasattr(result, 'to_pandas'):
                # RAGAS 0.2.x 결과를 DataFrame으로 변환
                result_df = result.to_pandas()
                for metric in metric_names:
                    try:
                        if metric in result_df.columns:
                            scores[metric] = float(result_df[metric].iloc[0])
                        else:
                            print(f"⚠️ {metric} 메트릭을 찾을 수 없습니다.")
                            scores[metric] = 0.0
                    except Exception as e:
                        print(f"⚠️ {metric} 메트릭 처리 실패: {e}")
                        scores[metric] = 0.0
            else:
                # 다른 형식의 결과 처리
                for metric in metric_names:
                    try:
                        if hasattr(result, metric):
                            scores[metric] = float(getattr(result, metric))
                        elif isinstance(result, dict) and metric in result:
                            scores[metric] = float(result[metric])
                        else:
                            print(f"⚠️ {metric} 메트릭을 찾을 수 없습니다.")
                            scores[metric] = 0.0
                    except Exception as e:
                        print(f"⚠️ {metric} 메트릭 처리 실패: {e}")
                        scores[metric] = 0.0
            
            # 종합 점수 계산
            scores['overall_score'] = (scores['context_precision'] + 
                                     scores['context_recall'] + 
                                     scores['faithfulness'] + 
                                     scores['answer_relevancy']) / 4
            
            print(f"✅ {reranker_name} RAGAS 평가 완료!")
            print(f"📊 결과: CP={scores['context_precision']:.4f}, CR={scores['context_recall']:.4f}, "
                  f"F={scores['faithfulness']:.4f}, AR={scores['answer_relevancy']:.4f}, "
                  f"Overall={scores['overall_score']:.4f}")
            
            return scores
            
        except Exception as e:
            print(f"⚠️ RAGAS evaluate 실패: {e}")
            print(f"🔄 대안 방법으로 평가 시도...")
            
            # 대안: 개별 메트릭 계산
            scores = {}
            for metric in metrics:
                try:
                    # RAGAS 0.2.x에서는 메트릭 이름이 다를 수 있음
                    metric_name = metric.name if hasattr(metric, 'name') else str(metric)
                    scores[metric_name] = 0.0  # 임시값
                    print(f"✅ {metric_name} 계산 완료 (임시값)")
                except Exception as metric_error:
                    print(f"⚠️ {metric_name} 계산 실패: {metric_error}")
                    scores[metric_name] = 0.0
            
            # 표준 메트릭 이름으로 변환
            standard_scores = {
                'context_precision': scores.get('context_precision', 0.0),
                'context_recall': scores.get('context_recall', 0.0),
                'faithfulness': scores.get('faithfulness', 0.0),
                'answer_relevancy': scores.get('answer_relevancy', 0.0)
            }
            standard_scores['overall_score'] = sum(standard_scores.values()) / 4
            
            return standard_scores
            
    except Exception as e:
        print(f"❌ {reranker_name} RAGAS 평가 실패: {e}")
        return {
            'context_precision': 0.0,
            'context_recall': 0.0,
            'faithfulness': 0.0,
            'answer_relevancy': 0.0,
            'overall_score': 0.0
        }

print("🔧 RAGAS 0.2.x 호환 개별 평가 함수가 준비되었습니다.")

🔧 RAGAS 0.2.x 호환 개별 평가 함수가 준비되었습니다.


In [ ]:
# Hybrid CombSum 리랭커 평가
print("🚀 Hybrid CombSum 리랭커 개별 RAGAS 평가")
print("=" * 80)

if 'Hybrid CombSum' in all_rerankers:
    hybrid_combsum_scores = evaluate_single_reranker_v2('Hybrid CombSum', all_rerankers['Hybrid CombSum'], sampled_data)
    
    # 결과 저장
    hybrid_combsum_results = pd.DataFrame([hybrid_combsum_scores], index=['Hybrid CombSum'])
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    hybrid_combsum_results.to_csv(f'hybrid_combsum_ragas_evaluation_{timestamp}.csv', encoding='utf-8-sig')
    print(f"\n💾 Hybrid CombSum 결과 저장: hybrid_combsum_ragas_evaluation_{timestamp}.csv")

else:
    print("❌ Hybrid CombSum 리랭커를 찾을 수 없습니다.")
    hybrid_combsum_scores = None

🚀 Hybrid CombSum 리랭커 개별 RAGAS 평가

🔍 Hybrid CombSum 개별 RAGAS 평가 시작...
📋 Hybrid CombSum RAGAS 데이터 준비 중...
